In [ ]:
import cv2
import dlib
import imutils
from imutils import face_utils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import os
%matplotlib inline

In [ ]:
PREDICTOR_FILE = "shape_predictor_68_face_landmarks.dat"
NUM_POINTS = 68
NUM_BASES = 20
FRAME_DIR = "frames"

In [ ]:
"""
Detects the largest face in the image and returns the facial landmarks. If no
face is detected, returns (None, None)
"""
def get_landmarks(image, detector, predictor):
    rects = detector(image, 1)
    if len(rects) == 0:
        return None, None

    largest_rect = max(rects, key=lambda x: x.area())
    shape = predictor(image, largest_rect)

    # Convert to friendly data types
    bounding_box = face_utils.rect_to_bb(largest_rect)
    shape = imutils.face_utils.shape_to_np(shape)
    return bounding_box, shape

"""
Extracts still frames from the video file every msecs milliseconds.
"""
def get_frames(video_path, msecs, max_frames=0, rotation=0):
    video = cv2.VideoCapture(video_path)
    _, frame = video.read()
    success = True
    timestamp = 0
    frames = []
    while success:
        video.set(cv2.CAP_PROP_POS_MSEC,timestamp)
        success, frame = video.read()
        if not success:
            break
        frame = imutils.rotate(frame, angle=rotation)
        frame = imutils.resize(frame, width=500)
        frames.append(frame)
        timestamp += msecs

        if max_frames > 0 and len(frames) == max_frames:
            break
    video.release()
    return frames

"""
Non-rigid factorization method as described in 'Recoverying Non-Rigid
3D Shape from Image Streams.

Arguments:
    points: 2T x P matrix of P points across T time steps
    K: the number of basis shapes
Returns:
    bases: basis shapes (3K x P)
    weights: basis weights (T x K)
    rotations: structure (T x 2 x 3)
"""
def nonrigid_sfm(points, K):
    # Get some parameter values.
    T = points.shape[0] / 2
    P = points.shape[1]
    rank = 3 * K

    # Center the points
    W = points - np.mean(points, axis=1, keepdims=True)

    # Apply SVD to decompose W into Q and B matrices
    Q, B = decomposition(W, rank)

    # Apply SVD to decompose Q into weights and rotations
    L = np.zeros((T, K))
    R = np.zeros((T, 2, 3))
    for t in range(T):
        q = Q[2*t:2*t+2, :] # 2 x 3K
        q_bar = np.zeros((K, 6)) # K x 6
        for k in range(K):
            q_bar[k, :] = q[:, 3*k:3*(k+1)].flatten()
        L_t, R_t = decomposition(q_bar, 1)
        L[t, :] = L_t.flatten()
        R[t, :, :] = R_t.reshape((2, 3))

    # Enforce orthonormality of rotation matrices
    A = np.zeros((3*T, 6))
    b = np.zeros((3*T))
    for t in range(T):
        r1, r2, r3, r4, r5, r6 = R[t, :, :].flatten()
        A[3*t, :] = np.array([r1*r1, 2*r1*r2, 2*r1*r3, r2*r2, 2*r2*r3, r3*r3])
        A[3*t+1, :] = np.array([r4*r4, 2*r4*r5, 2*r4*r6, r5*r5, 2*r5*r6, r6*r6])
        A[3*t+2, :] = np.array([r1*r4, r2*r4 + r1*r5, r3*r4 + r1*r6, r2*r5, r3*r5 + r2*r6, r3*r6])
        b[3*t:3*t+3] = np.array([1, 1, 0])
    Q, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    Q = np.array([[Q[0], Q[1], Q[2]],
                  [Q[1], Q[3], Q[4]],
                  [Q[2], Q[4], Q[5]]])
    G = np.linalg.cholesky(Q)
    G_inv = np.linalg.inv(G)
    R = np.matmul(R, G)
    for k in range(K):
        B[3*k:3*k + 3, :] = np.matmul(G_inv, B[3*k:3*k + 3, :])

    return B, L, R

def decomposition(A, rank):
    U, s, V = np.linalg.svd(A)
    s = np.sqrt(s[:rank])
    U = U[:, :rank] * s.reshape(1, rank)
    V = s.reshape(rank, 1) * V[:rank, :]
    return U, V

'''
SCATTER_3D_AXIS_EQUAL
Arguments:
    X - the coordinates on the x axis (N long vector)
    Y - the coordinates on the y axis (N long vector)
    Z - the coordinates on the z axis (N long vector)
    ax - the pyplot axis
Returns:
    Nothing; instead plots the points of (X, Y, Z) such that the axes are equal
'''
def scatter_3D_axis_equal(X, Y, Z, ax, xlims, ylims, zlims):
    ax.scatter(X, Y, Z)

    ax.set_xlim(*xlims)
    ax.set_ylim(*ylims)
    ax.set_zlim(*zlims)

"""
Arguments:
    points - 2 x P matrix of point coordinates
    bases - 3K x P
    weights - T x K weights
    rotations: structure - T x 2 x 3 rotation matrix
"""
def reprojection_error(points, bases, weights, rotations):
    T, K = weights.shape
    P = points.shape[1]
    error = 0.0
    points = points - np.mean(points, axis=1, keepdims=True)
    for t in range(T):
        structure = np.zeros((3, P)) # 3 x P
        for k in range(K):
            structure += weights[t, k] * bases[3*k : 3*(k + 1), :]

        reconstr_points = np.matmul(rotations[t, :, :], structure) # 2 x P
        error += np.linalg.norm(reconstr_points - points[2*t:2*t+2, :])
    return error / (T * P)

In [ ]:
if not os.path.exists(FRAME_DIR):
    os.makedirs(FRAME_DIR)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_FILE)

# No deformation pipeline

In [ ]:
print "Extracting frames from video3."
frames_nodef = get_frames("video3.mp4", 20, rotation=90) 
T_nodef = len(frames_nodef)
print "Extracted {} frames.".format(T_nodef)

In [ ]:
points_nodef = []
idx = 0
for t in range(T):
    _, landmarks = get_landmarks(frames_nodef[t], detector, predictor) # 68 x 2
    if landmarks is not None:
        points_nodef.append(landmarks[:, 0])
        points_nodef.append(landmarks[:, 1])

        cv2.imwrite(FRAME_DIR + '/original/video3_frame_{}_original.png'.format(idx), frames_nodef[t])
        for (x, y) in landmarks:
           cv2.circle(frames_nodef[t], (x, y), 5, (0, 0, 255), -1)
        cv2.imwrite(FRAME_DIR + '/landmarked/video3_frame_{}_landmarked.png'.format(idx), frames_nodef[t])
        idx += 1

print "Found a face in {} frames.".format(len(points_nodef) / 2)
points_nodef = np.array(points_nodef)

In [ ]:
bases, weights, rotations = nonrigid_sfm(points_nodef, 20) 

In [ ]:
print "Generating output frames."
T = weights.shape[0]
structure = np.zeros((T, 3, NUM_POINTS)) # T x 3 x P
for t in range(T):
    for k in range(1):
        structure[t, :, :] += weights[t, k] * bases[3*k : 3*(k + 1), :]

xlims = (np.min(structure[:, 0, :]), np.max(structure[:, 0, :]))
ylims = (np.min(structure[:, 1, :]), np.max(structure[:, 1, :]))
zlims = (np.min(structure[:, 2, :]), np.max(structure[:, 2, :]))

for t in range(T):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    scatter_3D_axis_equal(structure[t,0,:], 
                          structure[t,1,:], 
                          structure[t,2,:], 
                          ax, xlims, ylims, zlims)
    ax.set_title('frame_{}'.format(t))
    ax.view_init(80, 30)
    plt.savefig(FRAME_DIR + '/reconstruction/video3_frame_{}_reconstruction.png'.format(t))
    plt.close(fig)

print "Done."

In [ ]:
errors_nodef = []
for k in range(1, 21):
    bases, weights, rotations = nonrigid_sfm(points_nodef, k) 
    errors_nodef.append([reprojection_error(points_nodef, bases, weights, rotations)])
    
fig = plt.figure()
plt.plot(np.arange(1, 21), np.array(errors_nodef))
plt.xlabel("Number of basis shapes")
plt.ylabel("Mean reprojection error")
plt.title("Reprojection error without deformation")
plt.show()

# Deformation Pipeline

In [ ]:
print "Extracting frames from video4."
frames = get_frames("video.mp4", 20, rotation=90) 
T = len(frames)
print "Extracted {} frames.".format(T)

In [ ]:
points = []
idx = 0
for t in range(T):
    _, landmarks = get_landmarks(frames[t], detector, predictor) # 68 x 2
    if landmarks is not None:
        points.append(landmarks[:, 0])
        points.append(landmarks[:, 1])

        cv2.imwrite(FRAME_DIR + '/original/video_frame_{}_original.png'.format(idx), frames[t])
        for (x, y) in landmarks:
           cv2.circle(frames[t], (x, y), 5, (0, 0, 255), -1)
        cv2.imwrite(FRAME_DIR + '/landmarked/video_frame_{}_landmarked.png'.format(idx), frames[t])
        idx += 1

print "Found a face in {} frames.".format(len(points) / 2)
points = np.array(points)
#points = points - np.mean(points, axis=1, keepdims=True)

In [ ]:
print points

In [ ]:
bases, weights, rotations = nonrigid_sfm(points, 20) 

In [ ]:
print "Generating output frames."
T = weights.shape[0]
structure = np.zeros((T, 3, NUM_POINTS)) # T x 3 x P
for t in range(T):
    for k in range(1):
        structure[t, :, :] += weights[t, k] * bases[3*k : 3*(k + 1), :]

xlims = (np.min(structure[:, 0, :]), np.max(structure[:, 0, :]))
ylims = (np.min(structure[:, 1, :]), np.max(structure[:, 1, :]))
zlims = (np.min(structure[:, 2, :]), np.max(structure[:, 2, :]))

for t in range(T):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    scatter_3D_axis_equal(structure[t,0,:], 
                          structure[t,1,:], 
                          structure[t,2,:], 
                          ax, xlims, ylims, zlims)
    ax.set_title('frame_{}'.format(t))
    ax.view_init(80, 30)
    plt.savefig(FRAME_DIR + '/reconstruction/video_frame_{}_reconstruction.png'.format(t))
    plt.close(fig)
    
print "Done."

In [ ]:
errors = []
for k in range(1, 21):
    bases, weights, rotations = nonrigid_sfm(points, k) 
    errors.append(reprojection_error(points, bases, weights, rotations))
    
fig = plt.figure()
plt.plot(np.arange(1, 21).astype(np.int32), np.array(errors), 'g')
plt.xlabel("Number of basis shapes")
plt.ylabel("Mean reprojection error")
plt.title("Reprojection error with deformation")
plt.show()
print errors